In [245]:
# Import der benötigten Bibliotheken
import numpy as np  # Für numerische Berechnungen und Arrays
import random       # Für die zufällige Generierung von Werten

In [246]:
# Funktion zur Erstellung einer zufälligen Disjunktiven Normalform (DNF)
def create_random_dnf(n, m):
    """
    Erstellt eine zufällige DNF mit n Variablen und m Monomen.
    
    Parameter:
    - n: Anzahl der Variablen
    - m: Anzahl der Monome
    
    Rückgabe:
    - Ein numpy-Array, das die DNF repräsentiert. 
      Jedes Monom ist ein Array mit Werten -1, 1 oder 0.
    """
    dnf = []  # Liste zur Speicherung der Monome
    for i in range(m):
        monome = []  # Ein einzelnes Monom
        for j in range(n):
            # Zufällige Auswahl von -1, 1 oder 0 für jedes Literal
            monome.append(random.choice([-1, 1, 0]))
        dnf.append(monome)  # Monom zur DNF hinzufügen
    return np.array(dnf)  # Rückgabe als numpy-Array

# Test: Erstelle eine zufällige DNF mit 10 Variablen und 5 Monomen
test_dnf = create_random_dnf(10, 5)
print(test_dnf)  # Ausgabe der generierten DNF


[[ 0  1  1  0  0 -1  0 -1 -1  1]
 [ 0 -1  0  0  1  1  0  0  0 -1]
 [-1  0 -1 -1 -1  1 -1  1  1  0]
 [ 1  1  0 -1  1 -1 -1  1  1  1]
 [ 1  0 -1  1 -1 -1  0  0 -1 -1]]


In [247]:
# Signum-Funktion (sig), die die Werte eines Vektors auf -1 oder 1 setzt
def sig(x):
    """
    Wendet die Signum-Funktion auf jeden Wert eines Vektors an.
    
    Parameter:
    - x: Eingabevektor (numpy-Array)
    
    Rückgabe:
    - Ein numpy-Array mit Werten -1 oder 1.
    """
    ret = np.copy(x)  # Kopiere den Eingabevektor
    for i in range(np.shape(x)[0]):  # Iteriere über alle Elemente
        if x[i] >= 0:
            ret[i] = 1  # Setze positive Werte auf 1
        else:
            ret[i] = -1  # Setze negative Werte auf -1
    return ret.astype(np.int64)  # Rückgabe als Integer-Array

In [248]:
# Funktion zur Berechnung der Anzahl der verwendeten Literale in einem Monom
def get_used_literals(monome):
    """
    Zählt die Anzahl der nicht-null Werte in einem Monom.
    
    Parameter:
    - monome: Ein Array, das ein Monom repräsentiert
    
    Rückgabe:
    - Anzahl der verwendeten Literale (Werte ungleich 0).
    """
    literal_count = 0
    for elem in monome:
        if elem != 0:
            literal_count += 1  # Erhöhe den Zähler für jedes nicht-null Element
    return literal_count

In [249]:
# Funktion zur Berechnung der Zwischenwerte z
def calculate_z(w, x, v=None):
    """
    Berechnet die Zwischenwerte z basierend auf den Gewichten w und den Eingaben x.
    
    Parameter:
    - w: Gewichtsmatrix (numpy-Array), die die DNF repräsentiert
    - x: Eingabevektor (numpy-Array)
    
    Rückgabe:
    - Ein numpy-Array mit den berechneten z-Werten.
    """
    if v == None:
        v = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere v
        for i in range(w.shape[0]):
            v[i] = get_used_literals(w[i]) - 0.5  # Subtrahiere 0.5 von der Anzahl der Literale
    

    # Berechnung der z-Werte
    z = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere z
    for j in range(w.shape[0]):
        z[j] = np.dot(w[j], x) - v[j]  # Skalarprodukt minus v[j]
    z = sig(z)  # Wende die Signum-Funktion auf z an

    
    return z, w, v  # Rückgabe der z-Werte

# Test: Berechne z für die generierte DNF und einen Eingabevektor
z, v, w = calculate_z(test_dnf, np.array([1, -1, 1, -1, 1, -1, 1, 1, 1, 1]).astype(np.int64))

In [250]:
def calculate_y(z, **kargs):
    W = kargs.get("W", None)  # Standardwert für W
    V = kargs.get("V", None)  # Standardwert für V
    


    if W == None:
        W = np.ones_like(z, dtype=np.float64)
    
    if V == None:
        V = -(z.size - 1)

    y = np.matmul(W, z.T)
    
    y = y - V
    if y >= 0:
        y = 1
    else:
        y = -1
    return y, W, V
    
print(f"y: {calculate_y(z)}")

y: (-1, array([1., 1., 1., 1., 1.]), -4)


In [251]:
# Trainingsdaten
dnf_training = np.array([[-1, -1, -1,  1, -1,  1, -1,  1,  0, -1],
                        [-1,  0, -1,  1,  0, -1,  1,  1, -1,  0],
                        [ 1, -1,  1,  0, -1, -1, -1,  1,  0,  0],
                        [ 0, -1,  1,  0, -1, -1, -1,  1, -1, -1],
                        [-1, -1,  0,  1, -1,  1,  1,  0,  0, -1]]).astype(np.int64)


inputs_with_p = [
    (np.array([-1, -1, -1,  1, -1,  1, -1,  1,  1, -1]), 1),
    (np.array([-1,  1, -1,  1,  1, -1,  1,  1, -1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1,  1,  1]), 1),
    (np.array([ 1, -1,  1,  1, -1, -1, -1,  1, -1, -1]), 1),
    (np.array([-1, -1,  1,  1, -1,  1,  1,  1,  1, -1]), 1)
    
]



In [ ]:
def calculate_Epoch(input_x, p, eta, w=None, v=None, W=None, V=None):

    results_z = calculate_z(w, input_x, v)
    z, w, v = results_z
    y, W, V = calculate_y(z, W=W, V=V)
    error = p - y

    delta_W = np.copy(W)
    delta_V = np.copy(V)
    delta_v = np.copy(v)
    delta_w = np.copy(w).astype(np.float64)
    
    for j in range(delta_W.size):
        delta_W[j] = eta * error * z[j]
        delta_v[j] = -eta * error * W[j]
        for k in range(input_x.size):
            delta_w[j][k] = eta * W[j] * error * input_x[k]

    delta_V = -eta * error

    #Print Ausgabe
    print(f"y = {y} | p = {p} | Fehler: {error}")

    return delta_w, delta_v, delta_W, delta_V


In [253]:
print(calculate_Epoch(inputs_with_p[0][0], inputs_with_p[0][1], 0.1, w=dnf_training))

y = 1 | p = 1 | Fehler: 3
(array([[-0.3, -0.3, -0.3,  0.3, -0.3,  0.3, -0.3,  0.3,  0.3, -0.3],
       [-0.3, -0.3, -0.3,  0.3, -0.3,  0.3, -0.3,  0.3,  0.3, -0.3],
       [-0.3, -0.3, -0.3,  0.3, -0.3,  0.3, -0.3,  0.3,  0.3, -0.3],
       [-0.3, -0.3, -0.3,  0.3, -0.3,  0.3, -0.3,  0.3,  0.3, -0.3],
       [-0.3, -0.3, -0.3,  0.3, -0.3,  0.3, -0.3,  0.3,  0.3, -0.3]]), array([-0.3, -0.3, -0.3, -0.3, -0.3]), array([ 0.3, -0.3, -0.3, -0.3, -0.3]), -0.30000000000000004)


In [254]:
# Backpropagation-Funktion

def feedforwardBackproagation(w, trainings_data, epoches):
    
    input_training = trainings_data[0][0]
    p = trainings_data[0][1]
    eta = 0.1
    
    
    calculate_z(w, )

    for i in range(epoches):
        # Get random input from trainings Data
        trainings_data_cell = trainings_data[random.randint(0, len(inputs_with_p) - 1)]
        x_inputs = trainings_data_cell[0]
        p = trainings_data_cell[1]
        delta_w, delta_v, delta_W, delta_V = calculate_Epoch(x_inputs, p, eta, dnf_training)
        

In [255]:
# Funktion zur Berechnung der Zwischenwerte z
def calculate_z(w, x, v=None):
    """
    Berechnet die Zwischenwerte z basierend auf den Gewichten w und den Eingaben x.
    
    Parameter:
    - w: Gewichtsmatrix (numpy-Array), die die DNF repräsentiert
    - x: Eingabevektor (numpy-Array)
    - v: Schwellwerte (numpy-Array), optional
    
    Rückgabe:
    - Ein numpy-Array mit den berechneten z-Werten.
    """
    if v is None:
        v = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere v
        for i in range(w.shape[0]):
            v[i] = get_used_literals(w[i]) - 0.5  # Subtrahiere 0.5 von der Anzahl der Literale
    
    # Berechnung der z-Werte
    z = np.empty(w.shape[0], dtype=np.float64)  # Initialisiere z
    for j in range(w.shape[0]):
        z[j] = np.dot(w[j], x) - v[j]  # Skalarprodukt minus v[j]
    z = sig(z)  # Wende die Signum-Funktion auf z an
    
    return z, w, v  # Rückgabe der z-Werte

# Funktion zur Initialisierung der Gewichte und Schwellwerte
def initialize_weights_and_thresholds(dnf):
    w = np.copy(dnf).astype(np.float64)
    v = np.array([get_used_literals(m) - 0.5 for m in dnf], dtype=np.float64)
    W = np.ones(dnf.shape[0], dtype=np.float64)
    V = -(dnf.shape[0] - 1)
    return w, v, W, V

# Funktion zur Durchführung eines Tests
def test_implementation(dnf, inputs_with_p):
    w, v, W, V = initialize_weights_and_thresholds(dnf)
    for input_x, p in inputs_with_p:
        z, _, _ = calculate_z(w, input_x, v)
        y, _, _ = calculate_y(z, W=W, V=V)
        print(f"Input: {input_x}, Expected: {p}, Output: {y}")

# Funktion zur Durchführung einer Lernsequenz
def learning_sequence(dnf, inputs_with_p, epochs, eta=0.1):
    w, v, W, V = initialize_weights_and_thresholds(dnf)
    # Leichte Veränderung der Gewichte und Schwellwerte
    w += np.random.normal(0, 0.1, w.shape)
    v += np.random.normal(0, 0.1, v.shape)
    W += np.random.normal(0, 0.1, W.shape)
    V += np.random.normal(0, 0.1)
    
    for epoch in range(epochs):
        for input_x, p in inputs_with_p:
            delta_w, delta_v, delta_W, delta_V = calculate_Epoch(input_x, p, eta, w, v, W, V)
            w += delta_w
            v += delta_v
            W += delta_W
            V += delta_V
        print(f"Epoch {epoch + 1}/{epochs} completed")

    return w, v, W, V

# Test der Implementierung
print("Testing implementation with initial weights and thresholds:")
test_implementation(dnf_training, inputs_with_p)

# Lernsequenz starten
print("\nStarting learning sequence with slightly modified weights and thresholds:")
w, v, W, V = learning_sequence(dnf_training, inputs_with_p, epochs=10)

# Test nach der Lernsequenz
print("\nTesting implementation after learning sequence:")
test_implementation(dnf_training, inputs_with_p)


Testing implementation with initial weights and thresholds:


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()